In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../../Datasets/DecisionTrees/DecisionTreesDS/titanic.csv')

In [4]:
print(df.shape)

(891, 12)


In [5]:
df.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [6]:
col_to_drop = ["Name","Ticket","Embarked","PassengerId","Cabin"]

In [7]:
clean_data = df.drop(col_to_drop,axis=1)

In [8]:
clean_data.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le = LabelEncoder()

In [11]:
clean_data["Sex"] = le.fit_transform(clean_data["Sex"]) 

In [12]:
clean_data.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [13]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [14]:
clean_data = clean_data.fillna(clean_data["Age"].mean())

In [15]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [16]:
outpX = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
outpY = ["Survived"]

X = clean_data[outpX]
Y = clean_data[outpY]

In [17]:
print(X.shape,Y.shape)

(891, 6) (891, 1)


In [18]:
def entropy(col):
    
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    
    for i in counts[1]:
        p = float(i/N)       # prob 
        ent += (-1.0 * p * np.log2(p))
    return ent

In [19]:
col = np.array([1,1,1,0,1,1,1])
out = entropy(col)

In [20]:
def divide(data,fkey,fmean):
    
    # make Empty DF
    l = pd.DataFrame([],columns=data.columns)
    r = pd.DataFrame([],columns=data.columns)
    
    for i in range(data.shape[0]):
        val = data[fkey].loc[i]
        
        if val <= fmean:
            l = l.append(data.loc[i])
        else:
            r = r.append(data.loc[i])
    
    return l,r

In [21]:
pd.DataFrame?

In [22]:
# IG = H(s) - sum(x_prob * entropy(x))
# IG : information gain, H(s) : parent node entropy, x : distributions level such as left,right 
# x will be classified in binary
def information_gain(data,colName,data_mean):
    
    # for x_prob let's split/divide according to some criteria 
    # as we are doing binary class x_left : left dataframe satisfying data_mean and same for x_right
    x_left,x_right = divide(data,colName,data_mean)
    N = float(data.shape[0])
    
    left_prob = float(x_left.shape[0] / N)
    right_prob = float(x_right.shape[0] / N)
    
    if x_left.shape[0] == 0 or x_right.shape[0] == 0:
        return -10000000
    
    left_ent = entropy(x_left.Survived)
    right_ent = entropy(x_right.Survived)
    
    i_gain = entropy(data.Survived) - (left_prob * left_ent + right_prob * right_ent)
    return i_gain


In [23]:
col = X.columns

for curr_col in col:
    # checking which column is more suitable here passing entire data frame, col to check ig of, threshold for Binary Classification
    ig = information_gain(clean_data,curr_col,clean_data[curr_col].mean())
    print(curr_col)
    print(ig)

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.001158644038169343
SibSp
0.009584541813400071
Parch
0.015380754493137916
Fare
0.042140692838995464


In [24]:
print(X.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')


In [25]:
clean_data.loc[0]

Survived     0.00
Pclass       3.00
Sex          1.00
Age         22.00
SibSp        1.00
Parch        0.00
Fare         7.25
Name: 0, dtype: float64

In [26]:
class DecisionTree:
    
    # node structure contains left Df, right DF, key
    def __init__(self,depth=0,max_depth=5): # depth : current depth, max_depth : max depth that is acceptable
        
        self.left = None
        self.right = None
        self.fkey = None   #fkey : using which column distribution happened
        self.fval = None   #fval : some threshold criteria for Binary classification
        self.depth = depth
        self.max_depth = max_depth
        self.target = None  # target : making something majority changing its aim into single feature
        
    def train(self,X_train):
        
        features = list(X.columns)
        info_gain = []    # list to keep record of IG
        
        for i in features:
            ig = information_gain(X_train,i,X_train[i].mean())
            info_gain.append(ig)
        
        # now i have Info Gain for all feature select max 
        maxIndex = np.argmax(info_gain)
        self.fkey = features[maxIndex]
        self.fval = X_train[self.fkey].mean()
        print("Making Tree Feature is : ",self.fkey)
        
        # now divide by fkey
        x_left,x_right = divide(X_train,self.fkey,self.fval)  
        # index re-ordering is must
        x_left = x_left.reset_index(drop=True)
        x_right = x_right.reset_index(drop=True)
        
        # when will this end?
        # 1. reached Leaf, 2. reached maxdepth
        
        if x_left.shape[0] == 0 or x_right.shape[0] == 0:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        
        if (self.depth >= self.max_depth):
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        
        # RECURSIVE CASE
        # node -> left = solve()
        self.left = DecisionTree(depth = self.depth+1,max_depth = self.max_depth)
        self.left.train(x_left)
        
        self.right = DecisionTree(depth = self.depth+1,max_depth = self.max_depth)
        self.right.train(x_right)
        
        if X_train.Survived.mean() >= 0.5:
            self.target = "Survived"
        else:
            self.target = "Dead"
            
        return
    
    def predict(self,test):
    
        if test[self.fkey] > self.fval:
            # goto right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [27]:
DT = DecisionTree()
DT.train(clean_data)

Making Tree Feature is :  Sex
Making Tree Feature is :  Pclass
Making Tree Feature is :  Pclass
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  Fare
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Fare
Making Tree Feature is :  SibSp
Making Tree Feature is :  Fare
Making Tree Feature is :  Fare
Making Tree Feature is :  Age
Making Tree Feature is :  SibSp
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  SibSp
Making Tree Feature is :  Fare
Making Tree Feature is :  Parch
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree

## Train-Validation-test

In [28]:
split = int(0.7*clean_data.shape[0])
train_data = clean_data[:split]
test_data = clean_data[split:]
test_data = test_data.reset_index(drop=True)

print(train_data.shape,test_data.shape)
print(train_data.columns,test_data.columns)

(623, 7) (268, 7)
Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object') Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')


In [29]:
DT.train(test_data)

Making Tree Feature is :  Sex
Making Tree Feature is :  Pclass
Making Tree Feature is :  Pclass
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Parch
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Parch
Making Tree Feature is :  SibSp
Making Tree Feature is :  Parch
Making Tree Feature is :  Fare
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Age
Making Tree Feature is :  Pclass
Making Tree Feature is :  Age
Making Tree Feature is :  SibSp
Making Tree Feature is :  Age
Making Tree Feature 

In [30]:
print(DT.fkey)
print(DT.fval)
print(DT.left.fkey)
print(DT.right.fkey)

Sex
0.6902985074626866
Pclass
Fare


In [31]:
ypred = []

for ix in range(test_data.shape[0]):
    pred = DT.predict(test_data.loc[ix])
    ypred.append(pred)
ypred = np.array(ypred)

In [32]:
yact = Y[split:]
yact = np.array(yact)

le = LabelEncoder()
ypred = le.fit_transform(ypred)
ypred = ypred.reshape(-1,1)

acc = np.sum(ypred == yact)/ypred.shape[0]
#print(yact.shape,ypred.shape)

In [33]:
print(acc)

0.8731343283582089


## Decision Tree using Sklearn

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
dt = DecisionTreeClassifier(criterion='entropy',max_depth=5)

In [36]:
dt.fit(train_data[outpX],train_data[outpY])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [37]:
dt.predict(train_data[outpX])

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,

In [38]:
print(dt.score(test_data[outpX],test_data[outpY]))

0.8283582089552238
